In [1]:
%pip install --upgrade duckdb
%pip install --upgrade jupysql
%pip install --upgrade duckdb-engine

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.2/20.2 MB 2.7 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: duckdb
    Found existing installation: duckdb 1.1.2
    Uninstalling duckdb-1.1.2:
      Successfully uninstalled duckdb-1.1.2
Note: you may need to restart the kernel to use updated packages.
  Attempting uninstall: jupysql
    Found existing installation: jupysql 0.10.14
    Uninstalling jupysql-0.10.14:
      Successfully uninstalled jupysql-0.10.14
Note: you may need to restart the kernel to use updated packages.
  Attempting uninstall: duckdb-engine
    Found existing installation: duckdb_engine 0.13.4
    Uninstalling duckdb_engine-0.13.4:
      Successfully uninstalled duckdb_engine-0.13.4
Note: you may need to restart the kernel to use updated packages.


In [3]:
import duckdb
import pandas as pd
import os

In [29]:
base_path = '/home/fedepicado/'
modelos_path = base_path + 'buckets/b1/modelos/'
db_path = base_path + 'buckets/b1/db/'
datasets_path = base_path + 'buckets/b1/datasets/'
exp_path = base_path + 'buckets/b1/exp/'
dataset_file = 'competencia_02_crudo.csv'
full_path = datasets_path + dataset_file

# Verifica si el archivo existe
if os.path.exists(full_path):
    print("Archivo encontrado:", full_path)
else:
    print("Archivo no encontrado:", full_path)

Archivo encontrado: /home/fedepicado/buckets/b1/datasets/competencia_02_crudo.csv


In [5]:
conn= duckdb.connect(database=':memory:')

df= pd.read_csv(datasets_path + dataset_file)

In [5]:
a=df[df["foto_mes"]==201901]

In [6]:
a.shape

(124752, 154)

In [6]:
conn.register('competencia_02_crudo', df)

In [7]:
conn.execute("""
create or replace table competencia_02 as
with periodos as (
  select distinct foto_mes from competencia_02_crudo
), clientes as (
  select distinct numero_de_cliente from competencia_02_crudo
), todo as (
  select numero_de_cliente, foto_mes from clientes cross join periodos
), clase_ternaria as (
  select
  c.*
    , if(c.numero_de_cliente is null, 0, 1) as mes_0
  , lead(mes_0, 1) over (partition by t.numero_de_cliente order by foto_mes) as mes_1
  , lead(mes_0, 2) over (partition by t.numero_de_cliente order by foto_mes) as mes_2
  , case 
      when mes_2 = 1 then 'CONTINUA'
      when mes_1 = 1 and mes_2 = 0 then 'BAJA+2'
      when mes_1 = 0 and mes_2 = 0 then 'BAJA+1'
      when mes_1 = 0 and mes_2 = 1 then 'CONTINUA'
      else null
    end as clase_ternaria
  from todo t
  left join competencia_02_crudo c using (numero_de_cliente, foto_mes)
) 
select
* EXCLUDE (mes_0, mes_1, mes_2)
from clase_ternaria
where mes_0 = 1
""")

In [10]:
resultado = conn.execute("SELECT foto_mes, clase_ternaria FROM competencia_02").df()

In [16]:
conteo_clase_ternaria = resultado.groupby(['foto_mes', 'clase_ternaria']).size().reset_index(name='cantidad')
conteo_clase_ternaria.head(60)

,foto_mes,clase_ternaria,cantidad
0,201901,BAJA+1,9
1,201901,BAJA+2,3
2,201901,CONTINUA,1678
3,201902,BAJA+1,720
4,201902,BAJA+2,688
5,201902,CONTINUA,124391
6,201903,BAJA+1,688
7,201903,BAJA+2,760
8,201903,CONTINUA,124988
9,201904,BAJA+1,759


In [30]:
file_path = f"{datasets_path}/competencia_02.parquet" #### aca esta el tema

In [31]:
conn.execute(f"COPY competencia_02 TO '{file_path}' (FORMAT 'parquet')")